In [ ]:
########################################################################################
#
# Marcus Bischof
#
# Random Forest Classifier to Predict the likelihood that a given trip, originating
# from a given station, will end in a different neighborhood.
# 
# Why did I choose to predict this particular thing?
#
#    1) I imagine that a high priority for Divvy is to ultimately predict the capacity of racks & bikes
#       that should be stocked at each station.
#         1a) This is a difficult task, that may require significant effort to predict
#              1aa) I believe that more features (than provided..) would be required to accurately predict
#                   capacity. The divvy data that I have only provides the total
#                   capacity of a station as opposed to the total # of FREE racks.
#
#    2) I would like to contribute to predicting capacity, as I imagine an accurate prediction would likely
#       require an ensemble of models.
#         2a) THUS, I will build a model that will predict the liklihood that a trip ends in a different
#             neighborhood vs. the same neighborhood.
#              2aa) I will use a random forest classifier to do this, and I will use the neighborhood
#                   data that I acquired from the City of Chicago (https://data.cityofchicago.org/) to label
#                   the two classes.
#              2ab) This should benefit a larger model, because if this model predicts that a trip will
#                   end in another neighborhood with a high likelihood, one can assume that the meta-model
#                   would slightly increase the amount of future capacity at the station that this trip
#                   originated from.
#
########################################################################################

# The basics
import pandas as pd
import numpy as np
import pickle

# Modeling
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample

# Viz
from matplotlib import pyplot as plt

# Jupyter display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

df = pd.read_pickle('../data/interim/df_0_1000000.pkl')

In [ ]:
# Remove features that we wouldn't have at the start of a trip
for feature in [
    'stoptime', 'to_station_name', 'to_station_id', 'latitude_end', 'longitude_end',
    'dpcapacity_end', 'to_neighborhood', 'tripduration', 'same_station_trip'
]:
    del df[feature]

# Remove extraneous and/or non-predictive features (day, month, year, hour cover starttime)
for feature in [
    'trip_id', 'from_station_id', 'starttime'  
]:
    del df[feature]

In [ ]:
df.same_neighborhood_trip = df.same_neighborhood_trip.apply(lambda x : int(x))
df.from_neighborhood = df.from_neighborhood.astype('category')

# Change categorical columns to numerics
categoricals = [col for col in df.columns if df[col].dtype.name == 'category']
for categorical_column in categoricals:
    df[categorical_column] = df[categorical_column].cat.codes

In [ ]:
df.head()

In [ ]:
# Class that we are trying to predict: 1 -> same neighborhood trip, 2 -> different neighborhood trip
df.same_neighborhood_trip.value_counts()

In [ ]:
########################################################
#
# Deal with class imbalance, create train + test datasets
#
########################################################

# OH JOY. We have a class imbalance. Let's resample.
df_resample = pd.DataFrame()
df_same_hood = df[df.same_neighborhood_trip == 1]
df_diff_hood = df[df.same_neighborhood_trip == 0]

# Get length of same hood frame, we downsample df_diff_hood to this smaller length
num_same_hood = len(df_same_hood)

df_resample = resample(df_diff_hood, replace=False, n_samples=num_same_hood, random_state=0)

# Concatenate the frames
df_resample = pd.concat([df_resample, df_same_hood])

target = df_resample.pop('same_neighborhood_trip')
features = df_resample

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [ ]:
########################################################
#
# Random forest classifier.
# Cross Validation used to avoid overfitting.
# GridSearchCV used to tune hyper-parameters (yes, I've done minimal tuning here).
#
########################################################

rf_clf = RandomForestClassifier(class_weight='balanced',verbose=0,random_state=10,n_jobs=4)# instantiate model

n_estimators = np.linspace(10, 300, 6, dtype='int32')  # used to determine the number of estimators
max_features = [5, 7, 9, 11, 14]  # used to determine max number of features, we have a max of 14, but I would like 5 at the minimum

param_grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=[1,50])  # set up the grid for GridSearchCV
clf = GridSearchCV(rf_clf, param_grid=param_grid, cv=3, scoring='recall_macro')  # instantiate cross validation instance

clf.fit(X_train, y_train)  # train the classifier

# Test model
y_true, y_pred = y_test, clf.predict_proba(X_test)  # probability prediction for test set

In [ ]:
report = classification_report(y_test, clf.predict(X_test))  # create performance report

In [ ]:
print(report)

In [ ]:
########################################################
#
# Display top 15 features importances.
# Usefull when we have enough features to obscure view on bar chart.
#
########################################################

# Create plot of feature importance
feature_names = X_train.columns.values.tolist()
important_features = clf.best_estimator_.feature_importances_  # Important features

indices = np.argsort(important_features)
feature_names_ordered = [feature_names[i] for i in indices]

fig, ax1 = plt.subplots(1, 1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), important_features[indices], color='g', align='center')
plt.yticks(range(len(indices)), feature_names_ordered)
ax1.set_xticklabels
plt.xlabel('Relative Importance')

-  <b><font color='green'>Possibilities of overfitting</font></b>
   -  <b><font color='green'>hour + week + day --> temperature</font></b> (hour week and day <i>partly encode</i> temperature since various weeks are highly correlated with temperature). However, hour, week, and day are also significant because they determine things like: is it rush hour, is this a vacation week, is this a weekday vs. weekend, ...?
   -  <b><font color='green'>from_neighborhood --> latitude_start + longitude_start</font></b> (from_neighborhood only adds a little bit of extra pizzaz to latitude_start and longitude_end). It is obvious that a neighborhood is a polyline encircling (lat, long) tuples on a map, so in a way it is a more abstracted geocoded object. HOWEVER, as stated earlier a neighborhood has a vibe, spirit, and unique identity - it has a community. Residents gravitate towards neighborhoods in many cases due to these neighborhood identities. Some (lat, long) pairs may be close together, but residents biking from stations that are close but in different neighborhoods may treat bike trips differently. Thus, we will keep lat, long, AND from_neighborhood. HOWEVER, <b><font color='red'>from_station_name will be removed from the future model</font></b> since (lat, long) represents the orginating station.


Naturally, temperature is highly correlated with trip duration, and with the amount that humans sweat... It makes sense that this would help narrow down the liklihood that a long trip to another neighborhood will or will not occur. If it's february and freezing, I'm sure we are seeing fewer long inter-neighborhood trips.

In [ ]:
########################################################
#
# Standard deviation for the test score (precision) reported.
#
########################################################

print(clf.cv_results_['std_test_score'][list(clf.cv_results_['rank_test_score']).index(1)])

In [ ]:
# Save our model to a pickle, fitting this bad boy was intense.
filename = 'neighborhood_model_fitted_to_df_0_1000000.sav'
pickle.dump(rf_clf, open(filename, 'wb'))